<a href="https://colab.research.google.com/github/Lxyyxxx/Colab/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import io
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
from datetime import datetime
from typing import Any

In [ ]:
# tensorboard
# !rm -rf logs/mnist
logdir = "logs/mnist/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs/mnist

Reusing TensorBoard on port 6006 (pid 2550), started 0:53:16 ago. (Use '!kill 2550' to kill it.)

<IPython.core.display.Javascript object>

In [ ]:
batch_size = 32
lr = 0.1
epoch = 5

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# batch
train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

In [ ]:
class MnistModel(tf.keras.Model):
    def __init__(self) -> None:
        super().__init__()
        self.nn = [
            tf.keras.layers.Conv2D(16, (3, 3), input_shape=(28, 28, 1), activation='relu'),  # 无填充
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),  # 最大池化
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)), 
            tf.keras.layers.Dropout(0.25),  # 以 0.25 的概率选择神经元
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.25),
            tf.keras.layers.Dense(10, activation='softmax'),
        ]

    def call(self, x: Any) -> Any:
        for layer in self.nn:
            x = layer(x)
        return x

In [ ]:
def learn(model: MnistModel, x: Any, y: int) -> float:
    with tf.GradientTape() as tape:
        y_pred = model(x) # vector
        y = tf.one_hot(y, 10) # scale to vector
        loss = tf.reduce_mean(tf.square(y_pred - y)) # mse
    grads = tape.gradient(loss, model.trainable_weights)
    for v, g in zip(model.trainable_weights, grads): # gd
        v.assign_sub(g)
    return loss, accuracy_score(tf.argmax(y_pred, axis=1).numpy(), tf.argmax(y, axis=1).numpy())

In [ ]:
def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

def figure_grid(x, y):
    n = len(x)
    row = 4
    col = (n + row - 1) // row
    figure = plt.figure(1, figsize=(24, 12))
    plt.subplots_adjust(wspace=.5)
    plt.gray()
    for i in range(len(x)):
        plt.subplot(row, col, i + 1)
        img = x[i, :, :]
        plt.pcolor(255 - img)  # 图像
        plt.text(24.5, 26, '%d' % y_pred[i], color='cornflowerblue', fontsize=18)  # 标签
        plt.xlim(0, 27)
        plt.ylim(27, 0)
        plt.xticks([])
        plt.yticks([])
    return figure

In [ ]:
def test(model: MnistModel, x: Any, y: int):
    x = tf.expand_dims(x, -1)
    y_pred = tf.argmax(model(x), axis=1)
    return y_pred

In [ ]:
def run_episode(epoch: int, model: MnistModel) -> None:
    loss_all = []
    accuracy_all = []
    for (x_train, y_train) in train_db:
        x_train = tf.expand_dims(x_train, -1)
        loss, accuracy = learn(model, x_train, y_train)
        loss_all.append(loss)
        accuracy_all.append(accuracy)
    loss = sum(loss_all) / len(loss_all)
    accuracy = sum(accuracy_all) / len(accuracy_all)
    print("Epoch {}, loss: {}, accuracy: {}".format(epoch, loss, accuracy))
    # tensorboard
    with file_writer.as_default():
        tf.summary.scalar('loss', data=loss, step=epoch)
        tf.summary.scalar('accuracy', data=accuracy, step=epoch)

In [ ]:
if __name__ == '__main__':
    model = MnistModel()
    # train
    for e in range(epoch):
        run_episode(e, model)
    # test
    y_pred_all = []
    y_test_all = []
    for step, (x_test, y_test) in enumerate(test_db):
        y_pred = test(model, x_test, y_test)
        y_pred_all.extend(y_pred.numpy())
        y_test_all.extend(y_test)
        figure = figure_grid(x_test, y_pred)
        with file_writer.as_default():
            tf.summary.image('test_results', plot_to_image(figure), step=step)
    report = classification_report(y_pred_all, y_test_all, labels=[i for i in range(10)])
    print(report)
    with file_writer.as_default():
        tf.summary.text("classification_report", report, step=0)
    model.summary()

Epoch 0, loss: 0.012055537663400173, accuracy: 0.9154833333333333
Epoch 1, loss: 0.003087584860622883, accuracy: 0.98005
Epoch 2, loss: 0.002139296382665634, accuracy: 0.9864666666666667
Epoch 3, loss: 0.0016571555752307177, accuracy: 0.9896333333333334
Epoch 4, loss: 0.0013541723601520061, accuracy: 0.99155
